In [1]:
import numpy as np
from qutip import basis, tensor, sigmax, sigmay, sigmaz, expect
#DI-QKD
# Define Pauli matrices
sigma_x = sigmax()
sigma_y = sigmay()
sigma_z = sigmaz()

# Define computational basis states
up = basis(2, 0)    
down = basis(2, 1)  

# Create a maximally entangled Bell state (Bell state |Φ+⟩)
bell_state = (tensor(up, up) + tensor(down, down)).unit()

# Define measurement settings for Alice and Bob
# Alice's measurement angles (θ_A0 and θ_A1)
theta_A0 = 0
theta_A1 = np.pi / 4

# Bob's measurement angles (θ_B0 and θ_B1)
theta_B0 = np.pi / 8
theta_B1 = -np.pi / 8

# Function to create measurement operators
def measurement_operator(theta):
    return np.cos(theta) * sigma_z + np.sin(theta) * sigma_x

# Alice's measurement operators
A0 = measurement_operator(theta_A0)
A1 = measurement_operator(theta_A1)

# Bob's measurement operators
B0 = measurement_operator(theta_B0)
B1 = measurement_operator(theta_B1)

# Function to calculate expectation value
def expectation_value(operator, state):
    # Use the 'expect' function provided by QuTiP
    return np.real(expect(operator, state))

# Calculate the CHSH parameter S
def calculate_CHSH(state):
    AB0 = tensor(A0, B0)
    AB1 = tensor(A0, B1)
    A1B0 = tensor(A1, B0)
    A1B1 = tensor(A1, B1)
    
    S = (expectation_value(AB0, state) +
         expectation_value(AB1, state) +
         expectation_value(A1B0, state) -
         expectation_value(A1B1, state))
    return S

# Compute the CHSH parameter for the Bell state
S = calculate_CHSH(bell_state)
print(f"CHSH parameter S: {S}")

# Determine if Bell inequality is violated
if abs(S) > 2:
    print("Bell inequality is violated. Device-independent security is possible.")
else:
    print("Bell inequality is not violated.")


CHSH parameter S: 2.3889551651687704
Bell inequality is violated. Device-independent security is possible.


 In general, for a technological deployment of quantum
 cryptography and QKD, we will need to consider its in
tegration with the current classical infrastructure and
 develop layers of security, depending on the degree of
 confidentiality to be reached which, in turn, depends on
 the stakeholder and the type of business involved. 
 
 Protocols based on bounded-memories and quantum data
 locking provide a temporary low-level of quantum security that may be suitable for private personal communi
cations. Standard QKD protocols provide higher levels of
 security that may be suitable for financial transactions.
 Within QKD, different secret key rates might be consid
ered, for instance, with respect to individual, collective
 or fully-coherent attacks. The choice of these rates may
 also be associated with a specific sub-level of security to
 be reached. 
 
 Higher level of security, for applications such
 as political or strategic decisions, may involve the use of
 DI-QKD, which is more robust to both conventional and
 side-channel attacks. These aspects will become clearer
 and clearer as quantum cryptography will progressively
 become a wider technological product

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, tensor, sigmax, sigmay, sigmaz, expect, qeye
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets
from IPython.display import display

# Define Pauli matrices
sigma_x = sigmax()
sigma_y = sigmay()
sigma_z = sigmaz()

# Define computational basis states
up = basis(2, 0)    
down = basis(2, 1)  

# Create a maximally entangled Bell state (Bell state |Φ+⟩)
bell_state = (tensor(up, up) + tensor(down, down)).unit()

# Function to create measurement operators
def measurement_operator(theta):
    return np.cos(theta) * sigma_z + np.sin(theta) * sigma_x

# Function to calculate expectation value
def expectation_value(operator, state):
    # Use the 'expect' function provided by QuTiP
    return np.real(expect(operator, state))

# Function to calculate the CHSH parameter S
def calculate_CHSH(state, theta_A0, theta_A1, theta_B0, theta_B1):
    # Alice's measurement operators
    A0 = measurement_operator(theta_A0)
    A1 = measurement_operator(theta_A1)
    
    # Bob's measurement operators
    B0 = measurement_operator(theta_B0)
    B1 = measurement_operator(theta_B1)
    
    AB0 = tensor(A0, B0)
    AB1 = tensor(A0, B1)
    A1B0 = tensor(A1, B0)
    A1B1 = tensor(A1, B1)
    
    S = (expectation_value(AB0, state) +
         expectation_value(AB1, state) +
         expectation_value(A1B0, state) -
         expectation_value(A1B1, state))
    return S

# Function to define depolarizing channel Kraus operators
def depolarizing_channel(p):
    K0 = np.sqrt(1 - p) * qeye(2)
    K1 = np.sqrt(p / 3) * sigmax()
    K2 = np.sqrt(p / 3) * sigmay()
    K3 = np.sqrt(p / 3) * sigmaz()
    return [K0, K1, K2, K3]

# Function to apply depolarizing channel to a specific qubit
def apply_depolarizing_channel_to_qubit(rho_in, p, qubit_index, total_qubits):
    K_list = depolarizing_channel(p)
    rho_out = 0 * rho_in
    for K in K_list:
        # Create the operator for the full system
        operators = [qeye(2)] * total_qubits
        operators[qubit_index] = K
        K_full = tensor(operators)
        rho_out += K_full * rho_in * K_full.dag()
    return rho_out

# Apply depolarizing channel to both qubits
p = 0.1  # Depolarizing probability (adjust this value to simulate different noise levels)
rho_in = bell_state.proj()

# Apply depolarizing channel to Alice's qubit (qubit 0)
rho_noisy = apply_depolarizing_channel_to_qubit(rho_in, p, 0, 2)
# Apply depolarizing channel to Bob's qubit (qubit 1)
rho_noisy = apply_depolarizing_channel_to_qubit(rho_noisy, p, 1, 2)

# Interactive function
def interactive_CHSH(theta_A0, theta_A1, theta_B0, theta_B1):
    state = rho_noisy
    S = calculate_CHSH(state, theta_A0, theta_A1, theta_B0, theta_B1)
    print(f"CHSH parameter S: {S:.4f}")
    if abs(S) > 2:
        print("Bell inequality is violated. Device-independent security is possible.")
    else:
        print("Bell inequality is not violated.")
        
    # Plotting S as a function of one of the angles (e.g., theta_B0)
    thetas = np.linspace(-np.pi, np.pi, 200)
    S_values = []
    for theta in thetas:
        S_val = calculate_CHSH(state, theta_A0, theta_A1, theta, theta_B1)
        S_values.append(S_val)
    
    plt.figure(figsize=(8, 4))
    plt.plot(thetas, S_values, label='CHSH parameter S')
    plt.axhline(2, color='red', linestyle='--', label='Classical limit S=2')
    plt.axhline(-2, color='red', linestyle='--')
    plt.axvline(theta_B0, color='green', linestyle=':', label='Current θ_B0')
    plt.title('CHSH Parameter S vs θ_B0')
    plt.xlabel('θ_B0 (radians)')
    plt.ylabel('CHSH Parameter S')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive sliders for the measurement angles
theta_A0_slider = FloatSlider(value=0, min=-np.pi, max=np.pi, step=0.01, description='θ_A0')
theta_A1_slider = FloatSlider(value=np.pi/4, min=-np.pi, max=np.pi, step=0.01, description='θ_A1')
theta_B0_slider = FloatSlider(value=np.pi/8, min=-np.pi, max=np.pi, step=0.01, description='θ_B0')
theta_B1_slider = FloatSlider(value=-np.pi/8, min=-np.pi, max=np.pi, step=0.01, description='θ_B1')

# Display the interactive widgets
interact(interactive_CHSH,
         theta_A0=theta_A0_slider,
         theta_A1=theta_A1_slider,
         theta_B0=theta_B0_slider,
         theta_B1=theta_B1_slider);


interactive(children=(FloatSlider(value=0.0, description='θ_A0', max=3.141592653589793, min=-3.141592653589793…